In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType

spark=SparkSession.builder.appName('FordGoBike').getOrCreate()
spark

23/05/01 09:57:41 WARN Utils: Your hostname, codespaces-1e2092 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/05/01 09:57:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/01 09:57:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/01 09:57:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path="/workspaces/codespaces-jupyter/data/week_1_output.csv"
df_pyspark=spark.read.csv(path,header=True, inferSchema=True)

In [4]:
df_pyspark = df_pyspark.withColumn('start_time_timestamp', F.to_timestamp(F.concat_ws(':', 'start time hour', 'start time minute', 'start time seconds')))\
                        .withColumn('end_time_timestamp', F.to_timestamp(F.concat_ws(':', 'end_time hour', 'end_time minute', 'end_time seconds')))

In [5]:
df_pyspark=df_pyspark.withColumn('Diff_in_seconds',F.col('end_time_timestamp').cast('long')-F.col('start_time_timestamp').cast('long'))\
    .withColumn('Diff_in_minutes',(F.col('Diff_in_seconds')/60))\
    .withColumn('Trip_cost',(F.col('Diff_in_minutes')*0.35))

In [6]:
df_pyspark=df_pyspark.drop(*['end_time_timestamp','end_time_timestamp'],)
df_pyspark.show()

23/05/01 09:59:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+--------------------+---------------+------------------+------------------+
|start_time|start time hour|start time minute|start time seconds|end_time|end_time hour|end_time minute|end_time seconds|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|haversine_distance|start_time_timestamp|Diff_in_seconds|   Diff_in_minutes|         Trip_cost|
+----------+---------------+-----------------+------------------+--------+-------------+---------------+----------

In [9]:
df_pyspark.write.option("header",True).csv('/workspaces/codespaces-jupyter/data/week_2.csv')

In [8]:
df_pyspark.groupBy("bike_id").agg(F.sum("haversine_distance").alias("sum_distance")).sort(F.desc("sum_distance")).show(10)

+-------+-----------------+
|bike_id|     sum_distance|
+-------+-----------------+
|     68|743.3694820484669|
|   2178|721.1822432213559|
|    256|671.9160978829848|
|    235|670.1620031683858|
|   2049|656.8281017664256|
|    441| 656.642274625236|
|   2226|647.8232304063755|
|    796|646.8677297229265|
|    190| 640.294181741224|
|   2365|639.4126742864064|
+-------+-----------------+
only showing top 10 rows

